In [ ]:
# Config and imports

import pypsa
import logging
import atlite
import time
import warnings
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import plotly
from datetime import datetime
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets
from atlite.gis import shape_availability, ExclusionContainer
from lib.optimize import create_and_store_optimize
from lib.network import create_and_store_network
from lib.costs import create_and_store_costs
from lib.cutout import create_and_store_cutout
from lib.availability import create_and_store_availability
from lib.demand import create_and_store_demand
logging.basicConfig(level=logging.INFO)


# Configuration specified in /prototypes/configs
CONFIG_NAME = "single"
# In the config file, we have a secnarios configuration that is used to create files per scenario. In this notebook, we use the following scenario
SCENARIO = "weather=2023,costs=2030,demand=2023,load-target=19,network-nuclear=True,network-h2=True,network-biogas=Stor,network-onwind-limit=None,network-offwind-limit=None,geography=14"
# Check the config file, or print all possible options with following code:
# %run scenarios.py "list" "$CONFIG_NAME" False
%run scenarios.py "validate" "$CONFIG_NAME" False "$SCENARIO"

with open(f"../data/result/{SCENARIO}/config.json", "r") as f:
    CONFIG = json.load(f)


In [ ]:
# Setting variables based on config
DATA_PATH=CONFIG["scenario"]["data-path"]
WEATHER_START=CONFIG["scenario"]["weather_start"]
WEATHER_END=CONFIG["scenario"]["weather_end"]
LAN_CODE = CONFIG["scenario"]["geography_lan_code"]
KOM_CODE = CONFIG["scenario"]["geography_kom_code"]

## Wind turbines
WIND_TURBINE = CONFIG["onwind_turbine"]
WIND_TURBINE_OFFSHORE = CONFIG["offwind_turbine"]

## Cost assumptions
ASSUMPTIONS = pd.read_pickle(f"../{DATA_PATH}/costs.pkl")

## Cutout / selection
CUTOUT = atlite.Cutout(f"../{DATA_PATH}/cutout.nc")
SELECTION = gpd.read_file(f"../{DATA_PATH}/selection.shp")
EEZ = gpd.read_file(f"../{DATA_PATH}/eez.shp")
INDEX = pd.to_datetime(pd.read_csv(f"../{DATA_PATH}/time_index.csv")["0"])

# Cutout / availablity
AVAIL_SOLAR = xr.open_dataarray(f"../{DATA_PATH}/avail_solar.nc")
AVAIL_ONWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_onwind.nc")
AVAIL_OFFWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_offwind.nc")

AVAIL_CAPACITY_SOLAR = xr.open_dataarray(f"../{DATA_PATH}/avail_capacity_solar.nc")
AVAIL_CAPACITY_ONWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_capacity_onwind.nc")
AVAIL_CAPACITY_OFFWIND = xr.open_dataarray(f"../{DATA_PATH}/avail_capacity_offwind.nc")

DEMAND = pd.read_csv(f"../{DATA_PATH}/demand.csv", index_col=0, parse_dates=[0])

NETWORK = pypsa.Network()
NETWORK.import_from_netcdf(f"../{DATA_PATH}/network.nc")

STATISTICS = pd.read_pickle(f"../{DATA_PATH}/statistics.pkl")

parameters = pd.read_csv("../data/assumptions.csv")
parameters.set_index(['technology', 'parameter'], inplace=True)


In [ ]:
# Some weather availability / capacity graphs

fig = plt.figure(figsize=(18, 10))
gs = GridSpec(3, 3, figure=fig)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])

# Solar avail
AVAIL_SOLAR.sel(dim_0=0).plot(cmap="Reds", ax=ax1)
SELECTION.plot(ax=ax1, edgecolor="k", color="None")
CUTOUT.grid.plot(ax=ax1, color="None", edgecolor="grey", ls=":")
ax1.set_title("SOLAR AVAIL")

# Wind (onshore) avail
AVAIL_ONWIND.sel(dim_0=0).plot(cmap="Greens", ax=ax2)
SELECTION.plot(ax=ax2, edgecolor="k", color="None")
CUTOUT.grid.plot(ax=ax2, color="None", edgecolor="grey", ls=":")
ax2.set_title("WIND ONSHORE AVAIL")

# Wind (offshore) avail
AVAIL_OFFWIND.sel(dim_0=0).plot(cmap="Blues", ax=ax3)
SELECTION.plot(ax=ax3, edgecolor="k", color="None")
CUTOUT.grid.plot(ax=ax3, color="None", edgecolor="grey", ls=":")
ax3.set_title("WIND OFFSHORE AVAIL")

# Capacity factors
AVAIL_CAPACITY_SOLAR.plot(color="Red", ax=ax4)
AVAIL_CAPACITY_ONWIND.plot(color="Green", ax=ax5)
AVAIL_CAPACITY_OFFWIND.plot(color="Blue", ax=ax6)

print(AVAIL_CAPACITY_SOLAR.sum())
print(AVAIL_CAPACITY_ONWIND.sum())
print(AVAIL_CAPACITY_OFFWIND.sum())

In [ ]:
## OVERRIDING VARIABLES?
#CONFIG["demand-target"] = 0.05
# And then, to update any step, just uncomment and run
#create_and_store_costs(CONFIG)
#create_and_store_cutout(CONFIG)
#create_and_store_availability(CONFIG)
#create_and_store_demand(CONFIG)
#create_and_store_network(CONFIG)
#create_and_store_optimize(CONFIG)

STATISTICS


In [ ]:
# Exclude land use per solar/wind

# Source: https://www.uts.edu.au/oecm/renewable-resource-mapping
# (Classification mapping: https://collections.sentinel-hub.com/corine-land-cover/readme.html)
# With the exception of "41; Water bodies"

# Land-cover classes NOT included for solar areas:
EXCLUDED_SOLAR = [1,4,5,6,7,9,10,11,23,24,25,27,30,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
# Land-cover classes NOT included for wind energy areas:
EXCLUDED_WIND_NON_OCEAN = [1,2,3,4,5,6,7,9,10,11,15,16,17,23,24,25,27,30,31,33,34,35,36,37,38,39,40,41,42,43,44] # 1.1.1,1.1.2,1.2.1,1.2.2,1.2.3,1.2.4,1.3.1,1.3.3,1.4.1,1.4.2,2.2.1,2.2.2,2.2.3,3.1.1,3.1.2,3.1.3,3.2.2,3.3.1,3.3.2,3.3.4,3.3.5,4.1.1,4.1.2,4.2.1,4.2.2,4.2.3,5.1.1,5.1.2,5.2.1,5.2.2,5.2.3
INCLUDED_WIND_OCEAN = [44, 41] # 5.2.3, 5.1.2


CORINE = "../data/geo/corine.tif"
solar_excluder = ExclusionContainer()
onwind_excluder = ExclusionContainer()
offwind_excluder = ExclusionContainer()

solar_excluder.add_raster(CORINE, codes=EXCLUDED_SOLAR)
onwind_excluder.add_raster(CORINE, codes=EXCLUDED_WIND_NON_OCEAN)
offwind_excluder.add_raster(CORINE, codes=INCLUDED_WIND_OCEAN, invert=True)

solar_avail = CUTOUT.availabilitymatrix(SELECTION, solar_excluder)
onwind_avail = CUTOUT.availabilitymatrix(SELECTION, onwind_excluder)
offwind_avail = CUTOUT.availabilitymatrix(SELECTION, offwind_excluder)

In [ ]:
# Plot the dashboard
bus_colors = ["black", "yellow", "blue", "blue", "green", "red", "red", "red"]
network = NETWORK

## Parameters
window_size=56 # Rolling average window size for charts

## Create solar availability map widget
solar_map = widgets.Output()
with solar_map:
    solar_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Reds" })
    plt.show()

## Create onwind availability map widget
onwind_map = widgets.Output()
with onwind_map:
    onwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Greens" })
    plt.show()

## Create offwind availability map widget
offwind_map = widgets.Output()
with offwind_map:
    offwind_excluder.plot_shape_availability(SELECTION, show_kwargs = { "cmap": "Blues" })
    plt.show()

## Create solar capacity factor line chart widget
solar_capacity_line_chart = widgets.Output()
with solar_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_SOLAR.plot(ax=ax, color="Red", alpha=0.2)
    AVAIL_CAPACITY_SOLAR.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Red")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create onwind capacity factor line chart widget
onwind_capacity_line_chart = widgets.Output()
with onwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_ONWIND.plot(ax=ax, color="Green", alpha=0.2)
    AVAIL_CAPACITY_ONWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Green")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create offwind capacity factor line chart widget
offwind_capacity_line_chart = widgets.Output()
with offwind_capacity_line_chart:
    fig, ax = plt.subplots(figsize=(12, 4))
    AVAIL_CAPACITY_OFFWIND.plot(ax=ax, color="Blue", alpha=0.2)
    AVAIL_CAPACITY_OFFWIND.rolling(time=window_size, center=True).mean().plot(ax=ax, color="Blue")
    ax.set_ylabel("Capacity factor")
    ax.set_xlabel('')
    plt.show()

## Create target load line chart widget
load_line_chart = widgets.Output()
with load_line_chart:
    fig, ax = plt.subplots(figsize=(14, 7))
    DEMAND.plot(ax=ax, color='grey', alpha=0.2)
    DEMAND.rolling(window=window_size, center=True).mean().plot(ax=ax, color='blue', alpha=1)
    ax.legend(['Demand (3h average)', 'Demand (weekly average'])
    ax.set_ylabel("Load [MW]")
    ax.set_xlabel('')
    plt.show()


## Create network map widget
minx, miny, maxx, maxy = SELECTION.total_bounds
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy.mpl.feature_artist")
network_map = widgets.Output()
with network_map:
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    network.plot(ax=ax, boundaries = [minx-1, maxx+3, miny-1.5, maxy+4.5], color_geomap=True, bus_sizes=0.02, bus_colors=bus_colors)
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', label=bus, 
                            markerfacecolor=color, markersize=10) for bus, color in zip(network.buses.index, bus_colors)]
    plt.legend(handles=legend_labels, loc='upper left', bbox_to_anchor=(0.6, 0.7), ncol=1)
    plt.show()

## Format and create the parameters table widget
parameters_table = widgets.HTML(value=parameters.to_html())

## Format and create the system table widget widget
generator_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'p_nom_opt': 'Optimal Capacity'
}

store_column_names = {
    'capital_cost': 'Capital Cost',
    'marginal_cost': 'Marginal Cost',
    'lifetime': 'Lifetime',
    'e_nom_opt': 'Optimal Capacity'
}

generators = pd.concat([
    network.generators.loc[['Solar park', 'Onwind park', 'Offwind park']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']],
    network.links.loc[['Gas turbine']][['capital_cost', 'marginal_cost', 'lifetime', 'p_nom_opt']]
])

generators['capital_cost'] = generators['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000, 1)} MSEK/MW")
generators['marginal_cost'] = generators['marginal_cost'].apply(lambda x: f"{round(float(x), 1)} SEK/MWh")
generators['lifetime'] = generators['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
generators['p_nom_opt'] = generators['p_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MW")
generators.rename(columns=generator_column_names, inplace=True)

stores = network.stores.loc[["H2 storage", "Battery"]][['capital_cost', 'marginal_cost', 'lifetime', 'e_nom_opt']]
stores.loc[:, 'capital_cost'] = stores['capital_cost'].apply(lambda x: f"{round(float(x)/1_000_000,1)} MSEK/MWh")
stores.loc[:, 'marginal_cost'] = stores['marginal_cost'].apply(lambda x: f"{round(float(x),1)} SEK/MWh")
stores.loc[:, 'lifetime'] = stores['lifetime'].apply(lambda x: f"{int(round(float(x), 0))} years")
stores.loc[:, 'e_nom_opt'] = stores['e_nom_opt'].apply(lambda x: f"{int(round(float(x), 0))} MWh")
stores.rename(columns=store_column_names, inplace=True)

system_table = widgets.HTML(value=pd.concat([generators, stores]).to_html())


## Format and create the stores table widget widget

## Create generators size pie chart widget
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators['p_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Generator sizes')
    plt.show()

## Create stores size pie chart widget
store_pie_chart = widgets.Output()
with store_pie_chart:
    network.stores.loc[["H2 storage", "Battery"]]['e_nom_opt'].plot(figsize=(4, 4), kind='pie', labels=None, legend=True, startangle=90, title='Energy store sizes')
    plt.axis('off')  # Turns off the axis
    plt.show()

## Create price line chart widget
price_line_chart = widgets.Output()
with price_line_chart:
    network.buses_t.marginal_price.plot(figsize=(24, 6), legend=True, title='Marginal price')
    plt.show()

## Create generators output line chart widget
generator_line_chart = widgets.Output()
with generator_line_chart:
    fig, ax = plt.subplots(figsize=(16,6))
    network.generators_t.p.abs().plot(ax=ax, kind='area', stacked=True, legend=True, alpha=0.2, title='Power output')
    network.generators_t.p.abs().rolling(window=window_size, center=True).mean().plot(ax=ax, kind='area', stacked=True)
    plt.show()

## Create generators total output pie chart
generator_pie_chart = widgets.Output()
with generator_pie_chart:
    network.generators_t.p.sum().plot(figsize=(8, 6), kind='pie', labels=None, legend=True, title='Power output')
    plt.show()


## Create stores line chart widget
store_line_chart = widgets.Output()
with store_line_chart:
    network.stores_t.e[['H2 storage', 'Battery']].plot(figsize=(24, 6),legend=True, title='Energy stored')
    plt.show()


## Build the dashboard and show

sun_wind_map = widgets.HBox([solar_map, onwind_map, offwind_map])
sun_wind_cfs = widgets.VBox([solar_capacity_line_chart, onwind_capacity_line_chart, offwind_capacity_line_chart])

row_1 = widgets.HBox([widgets.VBox([load_line_chart, sun_wind_map]), sun_wind_cfs])

row_2 = widgets.HBox([widgets.VBox([network_map, system_table, widgets.HBox([generator_pie_chart, store_pie_chart])]), parameters_table])

row_3 = widgets.HBox([generator_line_chart, generator_pie_chart])

dashboard = widgets.VBox([row_1, row_2, row_3, store_line_chart, price_line_chart])

dashboard


In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
network.buses_t.marginal_price[['Main bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
network.stores_t.e[['H2 storage']].plot(ax=ax2,legend=True, color='red', title='Energy stored')


In [ ]:
fix, ax1 = plt.subplots(figsize=(24, 6))
                        
network.buses_t.marginal_price[['Main bus']].plot(ax=ax1, legend=True, title='Marginal price')

ax2 = plt.twinx(ax1)
DEMAND.plot(ax=ax2, color='grey', alpha=0.2)



In [ ]:
total_cost = network.statistics.capex().sum()
total_energy = network.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
total_cost = network.statistics.capex().sum()/30
total_energy = network.generators_t.p[['Solar park', 'Onwind park', 'Offwind park', 'Biogas market']].sum().sum()

total_cost/total_energy

In [ ]:
network.statistics.capex()

In [ ]:
4.3 * 5591

In [ ]:
network.statistics.capex()

In [ ]:
network.generators_t.p.abs().rolling(window=56, center=True).mean().plot(figsize=(16,6), kind='area', stacked=True)